In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# 그래프 기본 설정
plt.rcParams['font.family'] = 'Malgun Gothic'
# plt.rcParams['font.family'] = 'AppleGothic'
plt.rcParams['figure.figsize'] = 12, 6
plt.rcParams['font.size'] = 14
plt.rcParams['axes.unicode_minus'] = False

## 2023 데이터 전처리

In [3]:
df = pd.read_excel('data/2023/2023 외래관광객조사 DATA.xlsx')

- 국적: D_NAT
- 체류기간: M일HAP
- 동반자 유무: Q7A
- 동행자 유형: Q7a_dk ~ Q7a8
- 여행 형태: D_GUB
- 방문 목적: D_MOK
- 총지출: 총액1인TOT항공제외2
- 참여한 활동(자연경관 감상, 뷰티/미용 관광): Q8a03, Q8a12
- 참여 만족도: Q8_1a1, Q8_1a2, Q8_1a3
- 가중치: weight 
- 연령대 : D_AGE

In [4]:
cols_needed = ['D_NAT','M일HAP', 
               'Q7A','Q7a_dk', 'Q7a2', 'Q7a3', 'Q7a4', 'Q7a5', 'Q7a6', 'Q7a7', 'Q7a8',
               'D_GUB', 'D_MOK', '총액1인TOT항공제외2', 
               'Q8a03', 'Q8a12', 
               'Q8_1a1', 'Q8_1a2', 'Q8_1a3',
               'D_AGE']
df = df[cols_needed]

In [5]:
df.isnull().sum()

D_NAT               0
M일HAP               0
Q7A                 0
Q7a_dk          10690
Q7a2            13369
Q7a3            14923
Q7a4            14887
Q7a5            14512
Q7a6            12350
Q7a7            14476
Q7a8            16078
D_GUB               0
D_MOK               0
총액1인TOT항공제외2        0
Q8a03            7332
Q8a12           14576
Q8_1a1              0
Q8_1a2           1112
Q8_1a3           3149
D_AGE               0
dtype: int64

### 동행자 유형 분류

In [6]:
# 1. 동행자 관련 컬럼만 추출
companion_cols = ['Q7a_dk', 'Q7a2', 'Q7a3', 'Q7a4', 'Q7a5', 'Q7a6', 'Q7a7', 'Q7a8']

# 2. 결측치 처리 안함 
# 결측치는 <선택하지 않음>을 의미하므로 처리하지 않음

# 3. 동행자유형 파생 변수 만들기
## 1) 단순 그룹 (가족 여부 중심)
def classify_family_group(row):
    if row['Q7a_dk'] == 1:
        return '혼자'
    if any([row['Q7a2'] == 2, row['Q7a3'] == 3, row['Q7a4'] == 4, row['Q7a5'] == 5]):
        return '가족'
    if row['Q7a6'] == 6:
        return '친구'
    if row['Q7a7'] == 7:
        return '직장동료'
    if row['Q7a8'] == 8:
        return '기타'
    return '응답없음'

## 2) 세분화 그룹 (구체 동행자 중심)
def classify_detailed_group(row):
    if row['Q7a_dk'] == 1:
        return '혼자'
    if row['Q7a2'] == 2:
        return '배우자/파트너'
    if row['Q7a3'] == 3:
        return '부모님'
    if row['Q7a4'] == 4:
        return '자녀'
    if row['Q7a5'] == 5:
        return '그 외 가족 친지'
    if row['Q7a6'] == 6:
        return '친구'
    if row['Q7a7'] == 7:
        return '직장동료'
    if row['Q7a8'] == 8:
        return '기타'
    return '응답없음'

# 4. 적용
df['동행자_단순'] = df.apply(classify_family_group, axis=1)
df['동행자_세분화'] = df.apply(classify_detailed_group, axis=1)

# 5. 유효 응답만 남기기
## 두 변수 모두 '응답없음'이 아닌 행만 남김
df = df[(df['동행자_단순'] != '응답없음') & (df['동행자_세분화'] != '응답없음')].copy()

In [7]:
df.columns

Index(['D_NAT', 'M일HAP', 'Q7A', 'Q7a_dk', 'Q7a2', 'Q7a3', 'Q7a4', 'Q7a5',
       'Q7a6', 'Q7a7', 'Q7a8', 'D_GUB', 'D_MOK', '총액1인TOT항공제외2', 'Q8a03',
       'Q8a12', 'Q8_1a1', 'Q8_1a2', 'Q8_1a3', 'D_AGE', '동행자_단순', '동행자_세분화'],
      dtype='object')

In [8]:
df['동행자_단순'].value_counts()

동행자_단순
가족      5511
혼자      5506
친구      3526
직장동료    1588
기타        65
Name: count, dtype: int64

In [9]:
df['동행자_세분화'].value_counts()

동행자_세분화
혼자           5506
친구           3526
배우자/파트너      2827
직장동료         1588
부모님          1142
그 외 가족 친지     995
자녀            547
기타             65
Name: count, dtype: int64

### 국적 선별 
- 2023년 글로벌 방한 관광객 통계 기준 상위 6개국 
- GWI-Rankings 상위 5개국 중 2개국(방한관광객 상위 국가와 중복 제외)

In [10]:
df['D_NAT'].value_counts()

D_NAT
2     1963
1     1588
4     1444
3     1263
10     873
5      868
6      824
97     682
8      678
13     664
15     639
12     583
14     537
7      521
9      512
19     497
18     496
16     488
20     385
17     359
11     332
Name: count, dtype: int64

### 국가명 매핑

In [11]:
df['D_NAT'].dtype

dtype('int64')

In [12]:
nat_map = {
    1: '중국',
    2: '일본',
    3: '대만',
    4: '미국',
    5: '홍콩',
    6: '태국',
    7: '베트남',
    10: '싱가포르',
    16: '영국',
    18: '독일',
    12: '중동'
}

df['국가명'] = df['D_NAT'].map(nat_map)

# 분석 대상 국가만 필터링
df = df[df['국가명'].notna()]

In [13]:
df['국가명'].value_counts()

국가명
일본      1963
중국      1588
미국      1444
대만      1263
싱가포르     873
홍콩       868
태국       824
중동       583
베트남      521
독일       496
영국       488
Name: count, dtype: int64

### 연령대 분류

In [14]:
# 연령 통합 그룹
def classify_age_group_simple(code):
    if code == 1:
        return '10대'
    elif code == 2:
        return '20대'
    elif code in [3, 4, 5]:
        return '30~50대'
    elif code == 6:
        return '60대 이상'
    return '응답없음'

# 연령 세분화 그룹 (항목 정의서 그대로)
def classify_age_group_detailed(code):
    age_map = {
        1: '10대',
        2: '20대',
        3: '30대',
        4: '40대',
        5: '50대',
        6: '60대 이상'
    }
    return age_map.get(code, '응답없음')

# 적용
df['연령대_단순'] = df['D_AGE'].apply(classify_age_group_simple)
df['연령대_세분화'] = df['D_AGE'].apply(classify_age_group_detailed)

df = df[(df['연령대_단순'] != '응답없음') & (df['연령대_세분화'] != '응답없음')].copy()

In [15]:
df

,D_NAT,M일HAP,Q7A,Q7a_dk,Q7a2,Q7a3,Q7a4,Q7a5,Q7a6,Q7a7,...,Q8a12,Q8_1a1,Q8_1a2,Q8_1a3,D_AGE,동행자_단순,동행자_세분화,국가명,연령대_단순,연령대_세분화
0,3,27,1,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,8,1.0,2.0,2,혼자,혼자,대만,20대,20대
1,1,7,1,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,11,2.0,3.0,3,혼자,혼자,중국,30~50대,30대
2,4,2,1,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1,NaN,NaN,1,혼자,혼자,미국,10대,10대
3,5,8,1,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,15,4.0,16.0,5,혼자,혼자,홍콩,30~50대,50대
4,1,71,1,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,2,4.0,1.0,4,혼자,혼자,중국,30~50대,40대
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16191,1,6,2,NaN,NaN,NaN,NaN,NaN,6.0,NaN,...,12.0,11,10.0,12.0,4,친구,친구,중국,30~50대,40대
16192,1,5,2,NaN,NaN,NaN,NaN,NaN,6.0,NaN,...,NaN,3,2.0,1.0,3,친구,친구,중국,30~50대,30대
16193,1,4,2,NaN,NaN,NaN,NaN,NaN,6.0,NaN,...,NaN,3,4.0,9.0,3,친구,친구,중국,30~50대,30대
16194,1,4,2,NaN,NaN,NaN,NaN,NaN,6.0,NaN,...,NaN,3,1.0,2.0,4,친구,친구,중국,30~50대,40대


### 체류일수 및 총지출 0 이하 제거

In [17]:
df = df[(df['M일HAP'] > 0) & (df['총액1인TOT항공제외2'] > 0)]

### 웰니스 활동 참여 여부 파생변수 생성

In [18]:
df['Q8a03'].dtype

dtype('float64')

In [19]:
df['Q8a12'].dtype

dtype('float64')

In [20]:
df['웰니스참여'] = ((df['Q8a03'] == 3) | (df['Q8a12'] == 12)).astype(int)

In [21]:
df['웰니스참여'].value_counts()

웰니스참여
1    6078
0    4833
Name: count, dtype: int64

In [22]:
df['Q8_1a1'].value_counts()

Q8_1a1
2     2174
1     1985
3     1430
4     1156
17     797
7      605
11     425
5      355
6      351
10     309
12     269
18     229
13     189
19     170
15     123
8      111
16     106
14      64
9       63
Name: count, dtype: int64

In [23]:
df['Q8_1a2'].value_counts()

Q8_1a2
1.0     2843
2.0     2719
3.0     1361
4.0     1078
6.0      439
5.0      321
11.0     265
10.0     245
12.0     181
7.0      154
17.0     139
13.0      69
15.0      62
8.0       60
9.0       56
14.0      42
16.0      34
18.0      34
19.0       6
Name: count, dtype: int64

In [24]:
df['Q8_1a3'].value_counts()

Q8_1a3
2.0     1938
1.0     1678
3.0     1301
4.0     1055
6.0      513
11.0     370
5.0      366
10.0     289
12.0     264
7.0      187
17.0     149
13.0      89
8.0       84
9.0       69
15.0      61
18.0      43
14.0      37
16.0      23
19.0      13
Name: count, dtype: int64

- 자연경관(3)은 Top3 만족 활동 중 매우 자주 등장 (총 4000회 이상)

- 뷰티/미용(12)도 상대적으로 빈도는 적지만 분명히 선택되고 있음

### 웰니스 활동이 Top3 만족 활동에 포함된 총 응답자 수
- 현재는 각 코드별 총 등장 수만 계산됨 -> 실제 몇 명이 ‘3 또는 12’를 한번 이상 선택했는지 계산

In [25]:
df['웰니스만족'] = df[['Q8_1a1', 'Q8_1a2', 'Q8_1a3']].apply(
    lambda row: any(x in [3, 12] for x in row if pd.notnull(x)),
    axis=1
).astype(int)

df['웰니스만족'].value_counts()

웰니스만족
0    6223
1    4688
Name: count, dtype: int64

### 여행 목적 컬럼 (D_MOK) 전처리
- 여가/위락/힐링, 친구/친지 방문, 사업/전문 활동, 교육, 기타
- 여가/힐링을 즐길 의사와 여건이 있었던 응답자만 포함

In [26]:
df['D_MOK'].dtype

dtype('int64')

In [27]:
df = df[df['D_MOK'].isin([1, 2])]

In [28]:
df['D_MOK'].value_counts()

D_MOK
1    6799
2    1555
Name: count, dtype: int64

In [29]:
# 분류 성공 여부 및 결측 확인
# 결측치 확인
print("동행자_단순 결측치 수:", df['동행자_단순'].isnull().sum())

# 고유값 확인
print("동행자_단순 고유값 목록:", df['동행자_단순'].dropna().unique())

print("동행자_세분화 결측치 수:", df['동행자_세분화'].isnull().sum())
print("동행자_세분화 고유값 목록:", df['동행자_세분화'].dropna().unique())

동행자_단순 결측치 수: 0
동행자_단순 고유값 목록: ['혼자' '친구' '가족' '직장동료' '기타']
동행자_세분화 결측치 수: 0
동행자_세분화 고유값 목록: ['혼자' '친구' '자녀' '배우자/파트너' '직장동료' '부모님' '그 외 가족 친지' '기타']


In [30]:
# 분류 성공 여부 및 결측 확인
# 결측치 확인
print("연령대_단순 결측치 수:", df['연령대_단순'].isnull().sum())

# 고유값 확인
print("연령대_단순 고유값 목록:", df['연령대_단순'].dropna().unique())

print("연령대_세분화 결측치 수:", df['연령대_세분화'].isnull().sum())
print("연령대_세분화 고유값 목록:", df['연령대_세분화'].dropna().unique())

연령대_단순 결측치 수: 0
연령대_단순 고유값 목록: ['20대' '30~50대' '10대' '60대 이상']
연령대_세분화 결측치 수: 0
연령대_세분화 고유값 목록: ['20대' '30대' '40대' '50대' '10대' '60대 이상']


- ANOVA 검정 안정 확보를 위해 로그 변환 변수 추가

In [32]:
df['log_총액'] = np.log(df['총액1인TOT항공제외2'])

### 23년도 전처리 데이터 저장

In [34]:
df['year'] = 2023  # 연도 컬럼 붙이기
df.to_csv("data/2023/외래관광객조사_2023_전처리3(세분화).csv", index=False, encoding='utf-8-sig')